# Debugging using Large Language Models (LLMs) 2

The next code cell includes some code without comments or function/class description. The code is related to mobile robotics and ROS. Follow the same approach as in *Debugging using Large Language Models 1* to determine the following aspects:

- What functionality does the code implement?
- What do the parameters in lines 6-10 do?
- What do the cell's printed values mean?

*Hint: Try not to provide the LLM with the whole code at once. Current LLMs have trouble understanding long code segments. Alternatively, copy the code into the LLM's prompt in pieces.*

In [ ]:
import numpy as np
import time

publishToROS = False
dt = 0.1 # seconds
simulationTime = 20 # seconds
vt = 0.6 # m/s
wt = 0.8 # rad/s

def forwardKinematics(vt, wt, lastPose, dt, dtype=np.float32, addNoise=True):
    if not isinstance(lastPose, np.ndarray): 
        lastPose = np.array(lastPose, dtype=dtype)
        assert lastPose.shape == (3,), "Received pose in wrong format! Pose needs to be provided in form [x,y,theta]!"
    if wt == 0: wt = np.finfo(dtype).tiny
    if addNoise == True:
        wt *= 1 + (np.random.random()-0.5)/20 # -> Error between +/- 2.5%
        vt *= 1 + (np.random.random()-0.5)/20 # -> Error between +/- 2.5%
        dt *= 1 + (np.random.random()-0.5)/20 # -> Error between +/- 2.5%
    vtwt = vt/wt
    _, _, theta = lastPose
    return lastPose + np.array([
        -vtwt*np.sin(theta) + vtwt*np.sin(theta + (wt*dt)),
        vtwt*np.cos(theta) - vtwt*np.cos(theta + (wt*dt)),
        wt*dt
    ], dtype=dtype)

class simulator:
    def __init__(self):
        self.robotPose = np.array([0,0,0], dtype=np.float32)
        
        if publishToROS == True:
            pass

    def update(self, vt, wt):
        newPose = forwardKinematics(vt, wt, self.robotPose, dt)
        self.robotPose = newPose
        
        if publishToROS == True:
            pose_msg = PointStamped()
            pose_msg.header.frame_id = "map"
            pose_msg.point.x = self.robotPose[0]
            pose_msg.point.y = self.robotPose[1]
            self.pose_publisher.publish(pose_msg)

simulator_instance = simulator()

for i in range(int(simulationTime/dt)):
    simulator_instance.update(vt, wt)
    if i%int(1/dt) == 0: print(simulator_instance.robotPose)
    time.sleep(dt)

### Publishing to ROS

When you are comfortable with the code cell above, perform the following steps:

- Open a terminal and start ROS using the `roscore` command.
- Open a second terminal and start an RViz visualisation using `rosrun rviz rviz`.

Now change `publishToROS` in the code cell above to `True` and run the cell. It will cause an error. Ask the LLM what is wrong with the code and how to fix it. Pay special attention to what has been changed by the LLM, since LLMs tend to introduce new errors when changing longer code segments.

*Hint: You know that you are on the right track when the LLM adds required ROS imports.*

In [ ]:
# You can put the improved code in this cell



After having completed the code, you should be able to run the cell with `publishToROS` set to `True`. While the cell is running, click on the *Add* button in RViz and click on the *By topic* tab. Select the topic that you are publishing on (probably something like *robot_pose*) and click *OK*. You should now see the output of `forwardKinematics` visualised in ROS!

### Visualisation as Pose

After having successfully improved the code to publish to ROS, we will examine, that only two of the three calculated values are actually published to ROS (see lines 41 and 42 in this notebook's first code cell). In order to also publish the robot's rotation, the code has to be altered to publish a *PoseStamped* instead of a *PointStamped*. Use the LLM and alter the code so that the robot pose is published instead of the position.

*Hint: You will have to restart the roscore, RViz and the notebook kernel when changing the topic's message type. This is done by clicking on __Kernel__ $\to$ __Restart Kernel__ and stopping the terminals running roscore and RViz using the shortcut `Ctrl`+`c`.*


In [ ]:
# You can put the improved code in this cell



After having successfully altered the code, RViz will visualise an arrow indicating a simulated robot's pose. By default, the robot drives in a circle, however, you can change the linear (*vt*) and angular (*wt*) velocities and observe how the robot's trajectory changes.

If mobile robot kinematics have peaked your interest, please have a look at chapter 5 in S. *Thrun, (2005), Probabilistic Robotics, MIT Press*.